In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import mixed_precision
import os
import cv2
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
def load_images_from_directory(directory, label, target_size=(150, 150)):
    images = []
    labels = []
    for img_name in os.listdir(directory):
        img_path = os.path.join(directory, img_name)
        try:
            img = load_img(img_path, target_size=target_size)  # Resize image
            img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
            images.append(img_array)
            labels.append(label)
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
    return np.array(images), np.array(labels)

In [3]:
train_dir = r"C:\\Users\\71519\\Downloads\\ggg\\training_set\\training_set"

In [4]:
cats_train_images, cats_train_labels = load_images_from_directory(
    os.path.join(train_dir, "cats"), label=0
)

Error loading image C:\\Users\\71519\\Downloads\\ggg\\training_set\\training_set\cats\_DS_Store: cannot identify image file <_io.BytesIO object at 0x000001FC5433B290>


In [5]:
dogs_train_images, dogs_train_labels = load_images_from_directory(
    os.path.join(train_dir, "dogs"), label=1
)

Error loading image C:\\Users\\71519\\Downloads\\ggg\\training_set\\training_set\dogs\_DS_Store: cannot identify image file <_io.BytesIO object at 0x000001FC5C7E52B0>


In [6]:
X = np.concatenate((cats_train_images, dogs_train_images), axis=0)
y = np.concatenate((cats_train_labels, dogs_train_labels), axis=0)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
test_dir = r"C:\\Users\\71519\\Downloads\\ggg\\test_set\\test_set"

In [8]:
cats_test_images, cats_test_labels = load_images_from_directory(
    os.path.join(test_dir, "cats"), label=0
)

Error loading image C:\\Users\\71519\\Downloads\\ggg\\test_set\\test_set\cats\_DS_Store: cannot identify image file <_io.BytesIO object at 0x000001FC28865A30>


In [9]:
dogs_test_images, dogs_test_labels = load_images_from_directory(
    os.path.join(test_dir, "dogs"), label=1
)

Error loading image C:\\Users\\71519\\Downloads\\ggg\\test_set\\test_set\dogs\_DS_Store: cannot identify image file <_io.BytesIO object at 0x000001FC28864AE0>


In [10]:
X_test = np.concatenate((cats_test_images, dogs_test_images), axis=0)
y_test = np.concatenate((cats_test_labels, dogs_test_labels), axis=0)

In [33]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Step 12: Build the Keras neural network model with Conv2D layers
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    Dense(512, activation='relu'),
    
    Dense(2, activation='softmax')
])


In [34]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 64s 317ms/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 0.7865 - val_loss: 1.6189
Epoch 2/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 69s 342ms/step - accuracy: 1.0000 - loss: 1.2513e-04 - val_accuracy: 0.7914 - val_loss: 1.6605
Epoch 3/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 65s 325ms/step - accuracy: 1.0000 - loss: 8.5041e-05 - val_accuracy: 0.7904 - val_loss: 1.6943
Epoch 4/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 65s 325ms/step - accuracy: 1.0000 - loss: 6.2674e-05 - val_accuracy: 0.7899 - val_loss: 1.7209
Epoch 5/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 67s 331ms/step - accuracy: 1.0000 - loss: 5.2156e-05 - val_accuracy: 0.7909 - val_loss: 1.7457


In [41]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 6s 96ms/step - accuracy: 0.7884 - loss: 1.6727
Test Loss: 1.7457093000411987
Test Accuracy: 0.7909045815467834


In [42]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define image size and batch size
IMG_HEIGHT = 150
IMG_WIDTH = 150
BATCH_SIZE = 32

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [44]:
# Use ImageDataGenerator with validation split
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% of training data used for validation
)

# Training generator (80% of data)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'  # Use the training split
)

# Validation generator (20% of data)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'  # Use the validation split
)

Found 6404 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.


In [63]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [64]:
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_40 (Conv2D)              │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_40 (MaxPooling2D) │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_41 (MaxPooling2D) │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_42 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_42 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_43 (Conv2D)              │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_43 (MaxPooling2D) │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_11 (Flatten)            │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 512)            │     3,211,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,453,121 (13.17 MB)

 Trainable params: 3,453,121 (13.17 MB)

 Non-trainable params: 0 (0.00 B)

In [67]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


# Train the model
EPOCHS = 50
# Train the model with validation data
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=50
)

Epoch 1/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 144s 1s/step - accuracy: 0.6752 - loss: 0.6050 - val_accuracy: 0.7175 - val_loss: 0.5523
Epoch 2/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 701ms/step - accuracy: 0.7159 - loss: 0.5577 - val_accuracy: 1.0000 - val_loss: 0.0792
Epoch 3/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 156ms/step - accuracy: 0.8125 - loss: 0.5075 - val_accuracy: 0.7069 - val_loss: 0.5651
Epoch 4/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 66s 629ms/step - accuracy: 0.7006 - loss: 0.5856 - val_accuracy: 1.0000 - val_loss: 0.5008
Epoch 5/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 76s 760ms/step - accuracy: 0.7173 - loss: 0.5491 - val_accuracy: 0.7287 - val_loss: 0.5360
Epoch 6/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 150us/step - accuracy: 0.7500 - loss: 0.4670 - val_accuracy: 1.0000 - val_loss: 0.0703
Epoch 7/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 62s 609ms/step - accuracy: 0.7388 - loss: 0.5158 - val_accuracy: 0.7369 - val_loss: 0.5247
Epoch 8/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 62s 628ms/step - accuracy: 0.7196 - loss: 0.54

In [68]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 7s 100ms/step - accuracy: 0.8900 - loss: 0.2719
Test Loss: 0.30652475357055664
Test Accuracy: 0.8610973954200745
